In [3]:
import serial
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from IPython.display import display, clear_output
import time

# Configuration parameters - adjust these as needed
SERIAL_PORT = '/dev/ttyACM0'  # Change this to match your Arduino port (e.g., 'COM3' on Windows)
BAUD_RATE = 9600             # Make sure this matches the baud rate in your Arduino sketch
MAX_POINTS = 100             # Number of points to display on the plot

# Initialize the data arrays
times = np.array([])
values1 = np.array([])
values2 = np.array([])

# Set up the plot in interactive mode
plt.ion()
fig, ax = plt.subplots(figsize=(10, 6))
line1, = ax.plot([], [], 'r-', label='Value 1')
line2, = ax.plot([], [], 'b-', label='Value 2')

# Add labels and legend
ax.set_xlabel('Time (s)')
ax.set_ylabel('Values')
ax.set_title('Arduino Serial Data')
ax.legend()
ax.grid(True)

# Initialize the serial connection
try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    # Give the Arduino time to reset after opening the serial connection
    time.sleep(2)
    print(f"Connected to Arduino on {SERIAL_PORT}")
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    raise

# Start time
start_time = time.time()

# Function to update the plot
def update_plot():
    global times, values1, values2, start_time
    
    # Clear buffer of any previous data
    ser.reset_input_buffer()
    
    try:
        # Read data from serial
        if ser.in_waiting > 0:
            line = ser.readline().decode('utf-8').strip()
            
            # Parse the data (assuming values are space or comma separated)
            # Modify this parsing logic based on your Arduino's output format
            try:
                # Try comma-separated first
                if ',' in line:
                    data = line.split(',')
                    if len(data) >= 2:
                        val1 = float(data[0])
                        val2 = float(data[1])
                # Try space-separated if comma doesn't work
                else:
                    data = line.split()
                    if len(data) >= 2:
                        val1 = float(data[0])
                        val2 = float(data[1])
                
                # Calculate time elapsed
                current_time = time.time() - start_time
                
                # Append new data
                times = np.append(times, current_time)
                values1 = np.append(values1, val1)
                values2 = np.append(values2, val2)
                
                # Keep only the last MAX_POINTS
                if len(times) > MAX_POINTS:
                    times = times[-MAX_POINTS:]
                    values1 = values1[-MAX_POINTS:]
                    values2 = values2[-MAX_POINTS:]
                
                # Update the plot
                line1.set_data(times, values1)
                line2.set_data(times, values2)
                
                # Adjust plot limits
                ax.set_xlim(times.min(), times.max())
                y_min = min(values1.min() if len(values1) > 0 else 0, 
                           values2.min() if len(values2) > 0 else 0)
                y_max = max(values1.max() if len(values1) > 0 else 1, 
                           values2.max() if len(values2) > 0 else 1)
                margin = (y_max - y_min) * 0.1
                ax.set_ylim(y_min - margin, y_max + margin)
                
                # Redraw the plot
                fig.canvas.draw()
                fig.canvas.flush_events()
                
                # Print the latest values (optional)
                print(f"Time: {current_time:.2f}s, Value 1: {val1}, Value 2: {val2}")
                
            except (ValueError, IndexError) as e:
                print(f"Error parsing data: {line} ({e})")
    
    except Exception as e:
        print(f"Error reading from serial: {e}")

# Main loop for Jupyter notebook
try:
    print("Starting live plot. Press Ctrl+C to stop.")
    while True:
        update_plot()
        clear_output(wait=True)
        display(fig)
        # Adjust this delay to control how often the plot updates
        time.sleep(0.1)
except KeyboardInterrupt:
    print("Stopping live plot.")
finally:
    # Clean up
    ser.close()
    plt.ioff()
    plt.close(fig)

ModuleNotFoundError: No module named 'serial'